In [5]:
#import comet_ml in the top of your file
from comet_ml import Experiment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import seaborn as sns


df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_submission = pd.read_csv('sampleSubmission.csv')
df_train['datetime'] = pd.to_datetime(df_train['datetime'])
df_train['hour'] = df_train['datetime'].dt.hour
df_train['weekday'] = df_train['datetime'].dt.weekday
columnas_a_eliminar = ['weekday','datetime','humidity']
df_train = df_train.drop(columns=columnas_a_eliminar)

y_train = df_train['count']

X_train = df_train.drop(columns=['count','casual','registered'])
X_train.columns.tolist()


['season',
 'holiday',
 'workingday',
 'weather',
 'temp',
 'atemp',
 'windspeed',
 'hour']

In [8]:
from sklearn.metrics import mean_squared_log_error, make_scorer
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, np.maximum(0, y_pred)))

rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

In [9]:
from xgboost import XGBRegressor
param_grid = {
    "n_estimators": [50, 100, 200, 300],  
    "learning_rate": [0.01, 0.05, 0.1, 0.2]  
}

xgb_model = XGBRegressor(objective="reg:squarederror", random_state=2)


grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring=rmsle_scorer,
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)


best_model4 = grid_search.best_estimator_
best_params = grid_search.best_params_
best_rmsle = grid_search.best_score_  

In [ ]:
diccionario = {
    'max_depth': [5,8,10,12,15,20],
    'n_estimators' : [10,50,100,150,200,250,300]
    }

grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=2), param_grid=diccionario, cv=10)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_depth = grid_search.best_params_['max_depth']
best_n = grid_search.best_params_['n_estimators']

In [ ]:
diccionario2 = {
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 10, 15, 30]
    }


best_rf = RandomForestRegressor(max_depth=best_depth, n_estimators=best_n,n_jobs=-1,random_state=2)

grid_search2 = GridSearchCV(estimator=best_rf, param_grid=diccionario2, cv=10)

grid_search2.fit(X_train, y_train)

best_model2 = grid_search2.best_estimator_
best_leaf = grid_search2.best_params_['min_samples_leaf']
best_sample = grid_search2.best_params_['min_samples_split']

In [ ]:
best_params2 = grid_search2.best_params_

In [ ]:
y_casual = df_train['casual']
y_registered = df_train['registered']
X_casual = df_train.drop(columns=['count','casual','registered','temp'])
X_registered = df_train.drop(columns=['count','casual','registered','temp'])

In [ ]:
rf_model = RandomForestRegressor(**best_params2)
xgb_model = XGBRegressor(**best_xgb_params)

voting_model = VotingRegressor(estimators=[
    ("random_forest", rf_model),
    ("xgboost", xgb_model)
])


pipeline = Pipeline([
    ("scaler", StandardScaler()),  
    ("ensemble", voting_model)
])

In [ ]:
score_casual = cross_validate(pipeline,X_casual,y_casual,cv=5,scoring=rmsle_scorer)
score_registered = cross_validate(pipeline,X_registered,y_registered,cv=5,scoring=rmsle_scorer)

In [ ]:
print(score_casual['test_score'].mean()*-1)
print(score_registered['test_score'].mean()*-1)
print((score_casual['test_score'].mean()*-1 + score_registered['test_score'].mean()*-1)/2)